In [ ]:
!pip install git+https://github.com/Maluuba/nlg-eval.git@master -q
!pip install russian_paraphrasers -q

In [ ]:
import numpy as np 
import pandas as pd 
import os
from tqdm.auto import tqdm
from russian_paraphrasers import GPTParaphraser, Mt5Paraphraser

In [ ]:
paraphraser = Mt5Paraphraser(model_name="mt5-large", range_cand=True, make_eval=False)

In [ ]:
PATH_TO_DATA_FOLDER = "../input/kontyp-parsed-train-test"
train = pd.read_feather(os.path.join(PATH_TO_DATA_FOLDER,'train.feather'))

In [ ]:
def augment(sentence, max_augments):
    result = paraphraser.generate(
    sentence, n=10, temperature=1, 
    top_k=10, top_p=0.9, 
    max_length=100, repetition_penalty=1.5,
    threshold=0.7)['results'][0]['best_candidates']
    
    return result[:max_augments] if result else []

In [ ]:
NUM_AUGMENTATIONS = 1

result = list()
for i, row in tqdm(train.iterrows(), total = len(train)):
    augmented = augment(row.title, NUM_AUGMENTATIONS)
    for sentence in augmented:
        result.append((sentence, row.is_fake))
    result.append((row.title, row.is_fake))

In [ ]:
augmented_train = pd.DataFrame(result, columns = ['title', 'is_fake'])

In [ ]:
augmented_train.to_feather("augmented_train.feather")